### 1. 把德文評論全部翻譯成英文

In [ ]:
!pip install gdown
!gdown --id "1h8aZsJbKFowbRErlH5F8xrpXPXLuNegE"  
!gdown --id "12hgzs-_vviSx_ngSL2VKyA7CoG_VqwID"  
# !gdown --id "1d4Ee2J_FP39ZSLQZoElpOXYExQ5inmZc" 

Downloading...
From: https://drive.google.com/uc?id=1h8aZsJbKFowbRErlH5F8xrpXPXLuNegE
To: /content/reviews_summary.csv
100% 125M/125M [00:01<00:00, 109MB/s]
Downloading...
From: https://drive.google.com/uc?id=12hgzs-_vviSx_ngSL2VKyA7CoG_VqwID
To: /content/english_reviews_summary.csv
100% 234M/234M [00:01<00:00, 145MB/s]


In [ ]:
!pip install googletrans==4.0.0-rc1

     |████████████████████████████████| 55 kB 2.8 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 1.3 MB 18.0 MB/s 
     |████████████████████████████████| 65 kB 4.0 MB/s 
     |████████████████████████████████| 53 kB 2.5 MB/s 
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17415 sha256=14649b86a13db964be043fa2f07ac70e50856590af393bf670dfc151f1724663
  Stored in directory: /root/.cache/pip/wheels/43/34/00/4fe71786ea6d12314b29037620c36d857e5d104ac2748bf82a
Successfully built googletrans


In [ ]:
import pandas as pd
reviews_summary = pd.read_csv('reviews_summary.csv')  # 要翻譯的dataset

In [ ]:
reviews_summary.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,2015,69544350,2016-04-11,7178145,Rahel,Mein Freund und ich hatten gute gemütliche vie...
1,2015,69990732,2016-04-15,41944715,Hannah,Jan was very friendly and welcoming host! The ...
2,2015,71605267,2016-04-26,30048708,Victor,Un appartement tres bien situé dans un quartie...
3,2015,73819566,2016-05-10,63697857,Judy,"It is really nice area, food, park, transport ..."
4,2015,74293504,2016-05-14,10414887,Romina,"Buena ubicación, el departamento no está orden..."


In [ ]:
'''不要重跑這段'''
from googletrans import Translator
import time

reviews_list = reviews_summary['comments'].tolist()
# print(reviews_list)

translator = Translator()

english_comments = []
count = 0

for review in reviews_list:
  try:
    eng_comment = translator.translate(review, dest='en')
    print(eng_comment.text, count)
    english_comments.append(eng_comment.text)
    count += 1
  except:
    pass
    english_comments.append(' ')
    count += 1

  time.sleep(1)

In [ ]:
'''這段也千萬不要重跑~'''
english_comments_df = pd.DataFrame(english_comments)
english_comments_df.to_csv('english_reviews_summary.csv')

### 2. 用Afinn對評論進行情感分析

In [ ]:
reviews = pd.read_csv('english_reviews_summary.csv')

In [ ]:
reviews.head()

,Unnamed: 0,index,listing_id,id,date,reviewer_id,reviewer_name,comments,english_comments
0,0,0,2015,69544350,2016-04-11,7178145,Rahel,Mein Freund und ich hatten gute gemütliche vie...,My friend and I had good cozy four nights in J...
1,1,1,2015,69990732,2016-04-15,41944715,Hannah,Jan was very friendly and welcoming host! The ...,Jan was very friendly and welcoming host! The ...
2,2,2,2015,71605267,2016-04-26,30048708,Victor,Un appartement tres bien situé dans un quartie...,An apartment very well located in a very nice ...
3,3,3,2015,73819566,2016-05-10,63697857,Judy,"It is really nice area, food, park, transport ...","It is really nice area, food, park, transport ..."
4,4,4,2015,74293504,2016-05-14,10414887,Romina,"Buena ubicación, el departamento no está orden...","Good location, the department is not ordered, ..."


In [ ]:
reviews.english_comments[0]

'My friend and I had good cozy four nights in Jans Studio.It is incredibly quiet and even has a steamed piano for people who like to play.The location is excellent and there is a great alimentari and a cheap cultivation in the street.The bathroom is very old but functional and the insarked standard is ok, a little chaotic but you have the feeling you are at someone at home!Jan is an open-minded funny man and also tried for fresh towels and other!We thank.'

In [ ]:
# 刪掉不會用到的刪掉不會用到的columns
reviews.drop(['Unnamed: 0', 'index', 'id', 'date', 'reviewer_id', 'reviewer_name', 'comments'], axis=1, inplace=True)
print(reviews.shape)
reviews.head()

(401963, 2)


,listing_id,english_comments
0,2015,My friend and I had good cozy four nights in J...
1,2015,Jan was very friendly and welcoming host! The ...
2,2015,An apartment very well located in a very nice ...
3,2015,"It is really nice area, food, park, transport ..."
4,2015,"Good location, the department is not ordered, ..."


In [ ]:
!pip install afinn
from afinn import Afinn

     |████████████████████████████████| 52 kB 721 kB/s 
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53448 sha256=ea9398c10b9d1876baa4947442312b6d2e12d48fa2763f408d8dd15890af20e5
  Stored in directory: /root/.cache/pip/wheels/9d/16/3a/9f0953027434eab5dadf3f33ab3298fa95afa8292fcf7aba75
Successfully built afinn


In [ ]:
import numpy as np
afinn = Afinn(language='en')

def afinn_score(text):
    try:
        score = afinn.score(text)
        return score
    except:
        return np.nan

reviews['comments_afinn'] = reviews['english_comments'].apply(afinn_score)

In [ ]:
print(reviews.shape)
reviews.head()

In [ ]:
reviews = reviews[reviews["comments_afinn"].isnull()==False]
print(reviews.shape)

#### 計算每間房間Afinn score的平均分數

In [ ]:
afinn_list = reviews['comments_afinn'].tolist()
room_list = reviews['listing_id'].tolist()

score_dict = {}
for i in range(len(room_list)):
    if room_list[i] not in score_dict:
        score_dict[room_list[i]] = [afinn_list[i]]
    else:
        score_dict[room_list[i]].append(afinn_list[i])

print(len(score_dict))

In [ ]:
# 幫每個房間算房間算 afinn 的平均
for key, val in score_dict.items():
    score_dict[key] = round(sum(val) / len(val), 2)

In [ ]:
print(score_dict)
print(len(score_dict))

In [ ]:
# 每一間房間的平均Afinn分數
score_df = pd.DataFrame.from_dict(score_dict, orient='index').reset_index()
score_df.columns = ['Listing_id', 'afinn_avg']
score_df.head()

#### Output

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
score_df.to_csv('/content/drive/MyDrive/Colab Notebooks/統計學習與深度學習專案/文字處理/data_reviews.csv')

### 3. 串 listings_summary 跟 description
處理description text 的 code 再 Descriptions text 那份

In [ ]:
!gdown --id "1NadaQWvHuwmyqCN-9m-HTZJGKmu0RNtv"  # description 的 afinn 分數和和 PCA 們 
!gdown --id "1mjCpzl8CmYWvlsOeaygwGFV_sEjf3cqv"  # listings_summary_cleaned

#### Listings_summary_cleaned

In [ ]:
df = pd.read_csv('listings_summary_cleaned.csv')
df.head()

#### Description 分數 + PCA

In [ ]:
pd.set_option('display.max_columns', None)
description_score = pd.read_csv('data_des.csv')
print(description_score.shape)
description_score.head(1)

In [ ]:
# 只保留需要的欄位
use_cols = ['id', 'des_pc0',	'des_pc1',	'des_pc2',	'des_pc3',	'des_pc4',	'des_pc5',	'des_pc6',	'des_pc7',	'des_afinn_score']
description_score = description_score[use_cols]
print(description_score.shape)
description_score.head()

In [ ]:
# 串資料：listings_summary_cleaned + description
df_description = df.merge(description_score, left_on='id', right_on='id', how='left', right_index=False)

In [ ]:
# 檢查description有幾筆值
df_description.isna().sum()[df_description.isna().sum() > 0].to_frame(name="Num of NaN")

description 的 194 筆缺值統一補平均

In [ ]:
fill_cols = ['des_pc0', 'des_pc1', 'des_pc2', 'des_pc3', 'des_pc4', 'des_pc5', 'des_pc6', 'des_pc7', 'des_afinn_score']

for col in fill_cols:
    df_description[col].fillna(description_score[col].mean(), inplace=True)

In [ ]:
df_description.isna().sum()[df_description.isna().sum() > 0].to_frame(name="Num of NaN")  # 缺值都補上平均了

In [ ]:
print(df_description.shape)
print(df.shape)
df_description.head()  # 行數是正確的(多9行)

#### reviews 的 Afinn 分數 (上面的的score_df)

In [ ]:
score_df.head(3)

In [ ]:
# 串資料：df_description + score_df
df_description_reviews = df_description.merge(score_df, left_on='id', right_on='Listing_id', how='left', right_index=False)
df_description_reviews.drop('Listing_id', axis=1, inplace=True)

In [ ]:
df_description_reviews.isna().sum()[df_description_reviews.isna().sum() > 0].to_frame(name="Num of NaN")

reviews 的 3861 筆缺值統一補平均

In [ ]:
df_description_reviews['afinn_avg'].fillna(score_df['afinn_avg'].mean(), inplace=True)

In [ ]:
df_description_reviews.isna().sum()[df_description_reviews.isna().sum() > 0].to_frame(name="Num of NaN")  # 缺值都補上平均了

In [ ]:
print(df.shape)
print(df_description.shape)
print(df_description_reviews.shape)

df_description_reviews.head()  # 行數是正確的

Output

In [ ]:
df_description_reviews.to_csv('/content/drive/MyDrive/Colab Notebooks/統計學習與深度學習專案/文字處理/listings_summary_cleaned_text.csv')